## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 


In [538]:
import pandas as pd
data = pd.read_csv('data.csv', delimiter=',')
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [539]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Вывод

Перед нами список, состоящий из 12 столбцов и тысяч строк.

В строках встречаются различные типы данных, целые числа, вещественные и класс object. 

В столбце общий трудовой стаж: days employee  встречаются отрицательные значения, чего быть не должно

В столбце education значения забиты в разном регистре, что помешает дальнейшему анализу.

А в столбце purpose сразу бросается в глаза разичная формулировка целей: "приоберетние автомобиля" и "покупка автообил"я - это об одном и том же, поэтому необходимо найти все формлировки и привести их к одной общей.  

Так же с помощью метода .info(), я определила, что есть пропущенные значения в столбце с индексом 1 и 10.


### Шаг 2. Предобработка данных


In [540]:
##Замените названия столбцов (приведите к нижнему регистру);
##Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;
##Обработайте пропуски при необходимости:
##Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
##Опишите причины, которые могли привести к пропускам;
##Обратите внимание на аббревиатуру 'tbd' в столбцах с рейтингом. Отдельно разберите это значение и опишите, как его обработать;
##Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

Исследовала столбцы на нахождение данных "с ошибкой"

Так как в data['days_employed'] преобладающее большинство значений записаны со знаком минус, а отрицательного стажа работы быть не должно, нам следует изменить значения на положительные. 


А в столбце data['education'] привести все значения к одному регистру.

In [541]:
data['education'].value_counts()
data['education'] = data['education'].str.lower()

In [542]:
data['days_employed'].value_counts()

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

In [543]:
a = 0
for i in data['days_employed']:
    if i < 0:
        a += 1
    

In [544]:
a

15906

In [545]:
lenght_not_null_days_emp = len(data) - len(data[data['days_employed'].isna()])
percent_plus_days_emp = round(a / lenght_not_null_days_emp * 100, 2)
percent_plus_days_emp

82.2

Вывод: мы посчитали количество отрицательных значений с помощью цикла. 
Сохранили значение в переменной а, затем посчитали отношение количества отрицательных значений к количеству всех фреймов в сериусе.
Затем посчитали и сохранили значениче колличества заполненных фреймов, после того как вычли количество пропущенных значений из количества всех фремов
И нашли процентное отношение отрицательных значений к количеству заполненных фреймов в сериусе. 
Так как процент достаточно большой, я задумалась, может ли это быть ошибкой (человеческий фактор) или значения не верны и стоит их просто удалить. 

Кредитный скоринг - система оценки, основанная на численных статистических методах. Стаж работы важный критерий в оценке кредитоспособности лица, удалить эти значения без разрешения банка я не могу, потому воспользуюсь методом замены отрицательных значений на положительные


In [546]:
len(data.query('days_employed < 0 '))

15906

Замена циклу

In [547]:
data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [548]:

data['dob_years'].value_counts()
len(data.query('dob_years < 18 '))
#заемщик кредита в банке не может быть младше 18 лет, здесь ошибка

101

In [549]:
round(len(data.query('dob_years < 18 ')) / len(data) * 100, 2)


0.47

In [550]:
data = data[data['dob_years'] >= 18]
len(data.query('dob_years < 18'))

0

В процентном соотношении получается меньше 1% от всех значений датафрейма, было принято решение удалить данные

In [551]:
data['children'].value_counts()
#здесь следует заменить значения -1 и 20 на 1 и 20 соответственно, 
#скорее всего это опечатки

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [552]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2


In [553]:
data['children'].value_counts()

0    14080
1     4849
2     2117
3      328
4       41
5        9
Name: children, dtype: int64

In [554]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     19260 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      19260 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [555]:
data['education'].str.lower()
data['family_status'].str.lower()
data['gender'].str.lower()
data['income_type'].str.lower()
data['purpose'].str.lower()


0                       покупка жилья
1             приобретение автомобиля
2                       покупка жилья
3          дополнительное образование
4                     сыграть свадьбу
                     ...             
21520               операции с жильем
21521            сделка с автомобилем
21522                    недвижимость
21523    на покупку своего автомобиля
21524           на покупку автомобиля
Name: purpose, Length: 21424, dtype: object

##Вывод Таким образом мы:
нашли все пропущенные значения с помощью метода .isnull()и артефакты при помощи .value_counts() 
заменили пропуски при помощи .loc и удалили пропуски при помощи условия '>='. А так же привели к ниэнему регистру все сериусы с типом данных object


### Обработка пропусков

In [556]:
len(data[data['days_employed'].isna()])

2164

In [557]:
data['days_employed'] = abs(data['days_employed'])

In [558]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь стоит решить, что делать с пропущенными значениями, которых 2174! Найдем их процентное соотноение

In [559]:
round(len(data[data['days_employed'].isna()]) / len(data) * 100, 2)

10.1

10 % это немного, я скорее отнесу это к ошибке технического характера и заменю данные на средний стаж работы. Так как удаление данных приведет к их потере и, возможно, к ошибкам в выводах


In [560]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())


Мы заменили пропущенные значения в сериусе data['days_employed']. Так же пропущенные значения встречаются в сериусе total_income в таком же количестве у тех же людей, скорее это тоже ошибка технического характера, так как сферв занятости семейный статус, возраст клиентов разный и нет видимых причин, почему пропуски могли бы быть оставлены специально. Проведем с ним такую же операцию. И сделаем проверку таблицы методом isna() на наличие пропусков

In [561]:
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

In [562]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [563]:
data['gender'].value_counts()
#мы обнаружили один пропуск в граве "пол", так как он всего 1, не повлияет на резултат исследования

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

Пропущенных значений больше нет. Мы заполнили их средним значением по соответвующему столбцу, так как удаление 1/10 данных может привести к ошибке в выводе.


После обработки пропусков нам следует проверить другие столбцы на наличие артефактов. 

In [564]:
data['debt'].value_counts()

0    19691
1     1733
Name: debt, dtype: int64

In [565]:
data['total_income'].value_counts()

167465.224095    2164
89922.645314        1
133912.272223       1
184315.121979       1
182036.676828       1
                 ... 
115080.782380       1
84896.781597        1
153838.839212       1
177314.197183       1
150014.128510       1
Name: total_income, Length: 19261, dtype: int64

## Замена типа данных

In [566]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     21424 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      21424 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [567]:
#заменим тип float на int
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21424 non-null  int64 
 1   days_employed     21424 non-null  int64 
 2   dob_years         21424 non-null  int64 
 3   education         21424 non-null  object
 4   education_id      21424 non-null  int64 
 5   family_status     21424 non-null  object
 6   family_status_id  21424 non-null  int64 
 7   gender            21424 non-null  object
 8   income_type       21424 non-null  object
 9   debt              21424 non-null  int64 
 10  total_income      21424 non-null  int64 
 11  purpose           21424 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Вывод , изменили тип данных на целочисленный методом astype(int)

# Обработка дубликатов


In [568]:
data.duplicated().sum() #мы нашли 54 дубликата
#data

71

In [569]:
data = data.drop_duplicates().reset_index(drop = True)

In [570]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21353 entries, 0 to 21352
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21353 non-null  int64 
 1   days_employed     21353 non-null  int64 
 2   dob_years         21353 non-null  int64 
 3   education         21353 non-null  object
 4   education_id      21353 non-null  int64 
 5   family_status     21353 non-null  object
 6   family_status_id  21353 non-null  int64 
 7   gender            21353 non-null  object
 8   income_type       21353 non-null  object
 9   debt              21353 non-null  int64 
 10  total_income      21353 non-null  int64 
 11  purpose           21353 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Вывод. C помощью метода .duplicates() и .sum() мы проверили весь датафрейм на наличие полных дубликатов, 
метод value_counts() не подойдет, так как исследует значения только в одной колонке. Вызов info() проверил, удалились ли дубликаты 

# Лемматизация

In [571]:
Mystem

pymystem3.mystem.Mystem

In [572]:

m = Mystem()
lemmas = m.lemmatize('арбузный')
#lemmas = m.lemmatize(data.loc[data['purpose'][0:37]]lemmas = m.lemmatize(data.loc[data['purpose'][0:37]]')
#from collections import Counter
#print(Counter(lemmas))


In [573]:
data['lemma'] = data['purpose'].apply(lambda x: m.lemmatize(x))

In [574]:
data.head()



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [575]:
def listmerge(lstlst):
    all=[]
    for lst in lstlst:
        all.extend(lst)
    return all



In [576]:
lemmas_list = listmerge(data['lemma'])

In [577]:
from collections import Counter

In [578]:
Counter(lemmas_list).most_common()



[(' ', 33435),
 ('\n', 21353),
 ('недвижимость', 6328),
 ('покупка', 5870),
 ('жилье', 4436),
 ('автомобиль', 4284),
 ('образование', 3995),
 ('с', 2904),
 ('операция', 2593),
 ('свадьба', 2310),
 ('свой', 2223),
 ('на', 2210),
 ('строительство', 1873),
 ('высокий', 1366),
 ('получение', 1309),
 ('коммерческий', 1306),
 ('для', 1286),
 ('жилой', 1224),
 ('сделка', 938),
 ('дополнительный', 902),
 ('заниматься', 900),
 ('проведение', 764),
 ('сыграть', 760),
 ('сдача', 649),
 ('семья', 637),
 ('собственный', 633),
 ('со', 627),
 ('ремонт', 605),
 ('подержанный', 484),
 ('подержать', 478),
 ('приобретение', 459),
 ('профильный', 435)]

In [579]:
#for i in range(len(data['lemma'])):
#part = data[i][1]/emojixpress_total data[i].append(part)

In [580]:
def listmerge2(data):
    credit_purpose = []
    words = list(data['lemma'])
    for i in words:
        if 'автомобиль' in words:
            return 'автомобиль'
        if 'жилье' in words:
            return 'недвижимость'
        if 'недвижимость' in words:
            return 'недвижимость'
        if 'образование' in words:
            return 'образование'
        if 'свадьба'in words:
            return 'свадьба'
data['credit_purpose'] = data.apply(listmerge2, axis = 1)

In [581]:
#data['credit_purpose'] = data.apply(listmerge2, axis = 1)

In [582]:
data2 = data[['dob_years','education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'credit_purpose']]

In [583]:
data2.head(20)

,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,credit_purpose
0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


In [584]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
lemma               0
credit_purpose      0
dtype: int64

<!-- Провели лемматизацию датафрейма. 1 Шаг - вызвать словарь. 2- создан новый столбец lemma, который равен  data['purpose']? к которому применена функция lambda --> которая выполняет лемматизацию значений датафрейма purpose.
Затем мы применили функцию listmerge к столбцу с леммами, вызвали словарь Counter и смогли посмотреть на самые most_common() леммы в датафрейме.
На основе этого мне удалось создать цикл, где при помощи условия я смогла привести все цели кредита к общим, таким образом, можно переходить к категоризации данных

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.
- [x ]  открыт файл;
- [ x]  файл изучен;
- [ x]  определены пропущенные значения;
- [ x]  заполнены пропущенные значения;
- [ x]  есть пояснение какие пропущенные значения обнаружены;
- [ x]  описаны возможные причины появления пропусков в данных;
- [ x]  объяснено по какому принципу заполнены пропуски;
- [ x]  заменен вещественный тип данных на целочисленный;
- [ x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [ х]  удалены дубликаты;
- [ х]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [ х]  описаны возможные причины появления дубликатов в данных;
- [ х]  выделены леммы в значениях столбца с целями получения кредита;
- [ х]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.